In [1]:
import datetime
import numpy as np
import pandas as pd
import yfinance as yf
from math import sqrt
from pandas_datareader import data as pdr

In [2]:
df = pd.read_csv("stocksymbols - Sheet1.csv")
syms = list(df['Symbol'])
print(syms)

['SBIN.NS', 'AXISBANK.BO', 'IOC.BO', 'NMDC.BO', 'KRITIIND.BO', 'TATAPOWER.NS', 'AUROLAB.BO', 'RESONANCE.BO', 'ANDHRAPET.BO', 'MANALIPETC.BO', 'GAIL.NS', 'SAIL.NS', 'HDFCBANK.BO', 'ONGC.NS', 'INFY.BO', 'BHAGYAPROP.BO', 'FEDERALBNK.NS', 'WIPRO.BO', 'IDEA.NS', 'ALOKTEXT.BO', 'BANKBARODA.BO', 'BHEL.NS', 'DISHTV.BO', 'VEDL.BO', 'PTC.NS', 'PRESSMN.NS', 'VEDL.BO']


In [3]:
start = "2016-04-01"
end = str(int(start[:4]) + 3) + start[4:]
eq = []
for i in range(5):
    e = pdr.get_data_yahoo(syms[i], start = start, end = end)
    eq.append(e)
    e['Daily Return'] = e['Close'] - e['Open']

In [4]:
eq[0].head()

,High,Low,Open,Close,Volume,Adj Close,Daily Return
Date,,,,,,,
2016-04-01,197.250000,192.000000,193.699997,195.649994,21752206.0,191.382736,1.949997
2016-04-04,197.449997,192.550003,197.300003,194.699997,15732565.0,190.453461,-2.600006
2016-04-05,194.550003,183.300003,193.899994,184.350006,36635637.0,180.329224,-9.549988
2016-04-06,185.500000,182.199997,185.000000,183.850006,16350631.0,179.840118,-1.149994
2016-04-07,185.649994,181.000000,183.800003,181.949997,19430601.0,177.981552,-1.850006


In [5]:
for e in eq:
    e['Daily Return'] = (e['Close'] - e['Open'])*100 / e['Open']

In [6]:
for e in eq:
    avg = np.mean(e['Daily Return'])
    e['Ret - avg'] = e['Daily Return'] - avg

In [7]:
m = eq[0].shape[0]
n = len(eq)
excess_return = np.zeros([m, n])
for i in range(n):
    excess_return[:, i] = eq[i]['Ret - avg']

In [8]:
X = excess_return
mat = np.dot(np.transpose(X), X)

In [9]:
vcm = mat / m

In [10]:
std = []
for e in eq:
    mean = np.mean(e['Daily Return'])
    m = e.shape[0]
    st = sqrt(np.sum((e['Daily Return'] - mean)**2) / m)
    std.append(st)
print(std)

[1.876151865979658, 1.7354930090540837, 1.8297082602550838, 1.8422035518432398, 3.851188770588137]


In [11]:
stds = np.array(std).reshape(-1, 1)
print(stds.shape)

(5, 1)


In [12]:
product_std = np.dot(stds, np.transpose(stds))
product_std.shape

(5, 5)

### Correlation
**corr = variance_covariance_matrix / product_of_sds**

In [13]:
corr = vcm / product_std
print(corr)

[[1.         0.39331504 0.18812314 0.30136006 0.12007307]
 [0.39331504 1.         0.17336731 0.1810741  0.07055679]
 [0.18812314 0.17336731 1.         0.23930562 0.0635524 ]
 [0.30136006 0.1810741  0.23930562 1.         0.14940654]
 [0.12007307 0.07055679 0.0635524  0.14940654 1.        ]]


In [14]:
weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2]).reshape(-1, 1)
weights.shape

(5, 1)

In [15]:
weighted_sd = stds * weights
print(weighted_sd, stds)

[[0.37523037]
 [0.3470986 ]
 [0.36594165]
 [0.36844071]
 [0.77023775]] [[1.87615187]
 [1.73549301]
 [1.82970826]
 [1.84220355]
 [3.85118877]]


### Portfolio Variance
**Portfolio Variance = Sqrt (Transpose (Wt.SD) * Correlation Matrix * Wt. SD)**

In [16]:
portfolio_var = np.dot(np.dot(np.transpose(weighted_sd), corr), weighted_sd)
portfolio_var[0][0]

1.7442879620426066

In [17]:
final_pv = sqrt(portfolio_var[0][0])
final_pv

1.3207149435221086

### Maximizing Returns by fixing Portfolio Variance
#### Monte-Carlo Method: Using randomly generated weights
#### Monthly, Quarterly, Half-Yearly, Yearly Average Returns

In [18]:
monthly_avg = []
quarterly_avg = []
half_yearly_avg = []
yearly_avg = []
for e in eq:
    monthly = 0
    quarterly = 0
    half_yearly = 0
    yearly = 0
    # monthly
    start = "2016-04-01"
    for i in range(36):
        end = start[:5]
        month = int(start[5:7]) + 1
        if month <= 9:
            end += '0' + str(month) + start[7:]
        elif month > 12:
            end = str(int(start[:4]) + 1) + '-01-01'
        else:
            end += str(month) + start[7:]
        sliced_data = e.loc[(e.index >= start) & (e.index <= end)]
        monthly += (sliced_data.iloc[-1]['Close'] - sliced_data.iloc[0]['Open']) / sliced_data.iloc[0]['Open']
        start = end
    # quarterly
    start = "2016-04-01"
    for i in range(12):
        end = start[:5]
        month = int(start[5:7]) + 3
        if month <= 9:
            end += '0' + str(month) + start[7:]
        elif month == 13:
            end = str(int(start[:4]) + 1) + '-01-01'
        else:
            end += str(month) + start[7:]
        sliced_data = e.loc[(e.index >= start) & (e.index <= end)]
        quarterly += (sliced_data.iloc[-1]['Close'] - sliced_data.iloc[0]['Open']) / sliced_data.iloc[0]['Open']
        start = end
    # half-yearly
    start = "2016-04-01"
    for i in range(6):
        end = start[:5]
        month = int(start[5:7]) + 6
        if month == 16:
            end = str(int(start[:4]) + 1) + '-04-01'
        else:
            end += str(month) + start[7:]
        sliced_data = e.loc[(e.index >= start) & (e.index <= end)]
        half_yearly += (sliced_data.iloc[-1]['Close'] - sliced_data.iloc[0]['Open']) / sliced_data.iloc[0]['Open']
        start = end
    # yearly
    start = "2016-04-01"
    for i in range(3):
        end = str(int(start[:4]) + 1) + start[4:]
        sliced_data = e.loc[(e.index >= start) & (e.index <= end)]
        yearly += (sliced_data.iloc[-1]['Close'] - sliced_data.iloc[0]['Open']) / sliced_data.iloc[0]['Open']
        start = end
    monthly_avg.append(monthly*100/36)
    quarterly_avg.append(quarterly*100/12)
    half_yearly_avg.append(half_yearly*100/6)
    yearly_avg.append(yearly*100/3)

avgs = [monthly_avg, quarterly_avg, half_yearly_avg, yearly_avg]
print(f"Monthly Avg Returns: {monthly_avg}")
print(f"Quarterly Avg Return: {quarterly_avg}")
print(f"Half Yearly Avg Return: {half_yearly_avg}")
print(f"Yearly Avg Return: {yearly_avg}")

Monthly Avg Returns: [1.3864967281003655, 1.5220900943154332, 1.4226680341563298, 0.485194758014537, -0.9190434204316461]
Quarterly Avg Return: [4.8337968860190275, 5.1027703777242905, 5.2284894993381466, 0.6124079187611445, -1.1141471228940116]
Half Yearly Avg Return: [9.928907092626503, 9.412930512271943, 10.528820831093652, 0.6485528653539521, -0.7952194767056081]
Yearly Avg Return: [21.47260213692978, 21.24438064498441, 25.803607028716062, 3.570638202605776, 3.540235745283086]


In [19]:
for avg in avgs:
    random_weights = np.random.dirichlet(np.ones(5), size = 1).T
    total_return = avg * random_weights.T
    print(avg)
    print(random_weights.T)
    print(total_return)
    print(np.sum(total_return))

[1.3864967281003655, 1.5220900943154332, 1.4226680341563298, 0.485194758014537, -0.9190434204316461]
[[0.13882563 0.0732264  0.13213049 0.54854921 0.10726827]]
[[ 0.19248128  0.11145718  0.18797782  0.2661532  -0.0985842 ]]
0.6594852829572557
[4.8337968860190275, 5.1027703777242905, 5.2284894993381466, 0.6124079187611445, -1.1141471228940116]
[[0.13731262 0.04156266 0.03773539 0.09721468 0.68617465]]
[[ 0.66374133  0.21208469  0.19729908  0.05953504 -0.76449951]]
0.3681606323571027
[9.928907092626503, 9.412930512271943, 10.528820831093652, 0.6485528653539521, -0.7952194767056081]
[[0.20322202 0.42841738 0.07988923 0.09466311 0.19380826]]
[[ 2.01777256  4.03266303  0.84113935  0.06139403 -0.1541201 ]]
6.798848865114897
[21.47260213692978, 21.24438064498441, 25.803607028716062, 3.570638202605776, 3.540235745283086]
[[0.0170565  0.0134307  0.38624071 0.11225079 0.4710213 ]]
[[0.36624749 0.28532689 9.96640349 0.40080697 1.66752643]]
12.686311260837856


#### Annual Return

In [20]:
ann_ret = []
for e in eq:
    ret = (e.iloc[-1]['Close'] - e.iloc[0]['Open'])*100 / e.iloc[0]['Open']
    ann_ret.append(ret)
annual_return = np.array(ann_ret).reshape(-1, 1)
for avg in avgs:
    random_weights = np.random.dirichlet(np.ones(5), size = 1).T
    total_return = annual_return.T * random_weights.T
    print(annual_return.T)
    print(random_weights.T)
    print(total_return)
    print(np.sum(total_return))

[[ 66.62364744  72.45576155  62.03303064   5.30265454 -13.67127177]]
[[1.51571350e-01 1.17332518e-04 2.20479811e-01 4.52997402e-01
  1.74834104e-01]]
[[ 1.00982362e+01  8.50141698e-03  1.36770309e+01  2.40208873e+00
  -2.39020455e+00]]
23.795652672030528
[[ 66.62364744  72.45576155  62.03303064   5.30265454 -13.67127177]]
[[0.37970635 0.12878146 0.39448123 0.03331765 0.0637133 ]]
[[25.29742219  9.33095911 24.4708665   0.17667199 -0.87104181]]
58.404877985528834
[[ 66.62364744  72.45576155  62.03303064   5.30265454 -13.67127177]]
[[0.4095546  0.26357323 0.03865543 0.28450087 0.00371587]]
[[27.28602139 19.09739902  2.39791342  1.50860981 -0.05080073]]
50.239142911758414
[[ 66.62364744  72.45576155  62.03303064   5.30265454 -13.67127177]]
[[0.12722201 0.197121   0.39291465 0.08015447 0.20258788]]
[[ 8.47599424 14.2825519  24.37368668  0.42503145 -2.76963391]]
44.78763036428646


In [21]:
types = ['Monthly', 'Quarterly', 'Half-Yearly', 'Yearly']
best_wts_for_avgs = []
annual_returns_for_avgs = []
port_vars_for_avgs = []
for avg in avgs:
    min_port_var = 1.22
    max_port_var = 1.27
    port_vars = []
    returns = []
    rand_wts = []
    for i in range(0, 20000):
        random_weights = np.random.dirichlet(np.ones(5), size = 1).T
        random_weighted_sd = stds * random_weights
        portfolio_var = sqrt(np.sum(np.dot(np.dot(np.transpose(random_weighted_sd), corr), random_weighted_sd)))
        if (portfolio_var >= min_port_var) & (portfolio_var <= max_port_var):
            port_vars.append(portfolio_var)
            rand_wts.append(random_weights)
            total_return = np.sum(avg * random_weights.T)
            returns.append(total_return)
    max_index = returns.index(max(returns))
    max_return = returns[max_index]
    best_wts = rand_wts[max_index]
    min_var = port_vars[max_index]

    best_wts_for_avgs.append(best_wts)
    port_vars_for_avgs.append(min_var)
    annual_ret = np.sum(best_wts * annual_return)
    annual_returns_for_avgs.append(annual_ret)

max_index = annual_returns_for_avgs.index(max(annual_returns_for_avgs))
maximum_return = annual_returns_for_avgs[max_index]
bestest_wts = best_wts_for_avgs[max_index]
miniest_var = port_vars_for_avgs[max_index]
print(f"Best Performance: {types[max_index]}")
print(f"Best Return: {maximum_return}, Minimum Portfolio Variance: {miniest_var}")
print(f"Best Weights: {bestest_wts}, Sum of weights: {np.sum(bestest_wts)}")

Best Performance: Quarterly
Best Return: 66.02045762161664, Minimum Portfolio Variance: 1.269802149185382
Best Weights: [[0.19481494]
 [0.42967363]
 [0.35593298]
 [0.00510736]
 [0.01447109]], Sum of weights: 0.9999999999999999


In [22]:
ordered_types = {}
for i in range(4):
    ordered_types[types[i]] = annual_returns_for_avgs[i]
print(ordered_types)

{'Monthly': 65.29814113885729, 'Quarterly': 66.02045762161664, 'Half-Yearly': 65.39435402126622, 'Yearly': 65.16165703787783}


### Minimizing Portfolio Variance

#### Monte-Carlo Method: Using randomly generated weights

In [23]:
port_vars = []
rand_wts = []
for i in range(0, 10000):
    random_weights = np.random.dirichlet(np.ones(5), size = 1).T
    rand_wts.append(random_weights)
    random_weighted_sd = stds * random_weights
    portfolio_var = sqrt(np.sum(np.dot(np.dot(np.transpose(random_weighted_sd), corr), random_weighted_sd)))
    port_vars.append(portfolio_var)
min_index = port_vars.index(min(port_vars))
best_wts = rand_wts[min_index]
min_var = port_vars[min_index]
net_return = annual_return * best_wts
print(f"Best Weights: {best_wts}, Minimum Portfolio Variance: {min_var}, Sum of weights: {np.sum(best_wts)}")
print(f"Return: {np.sum(net_return)}")

Best Weights: [[0.14222324]
 [0.30742436]
 [0.25680305]
 [0.25962734]
 [0.03392201]], Minimum Portfolio Variance: 1.17903482967811, Sum of weights: 0.9999999999999999
Return: 48.59332608945021


### Maximizing Sharpe Ratio

#### Monte-Carlo Method: Using randomly generated weights

In [24]:
start = "2016-04-01"
end = str(int(start[:4]) + 1) + start[4:]
e = pdr.get_data_yahoo("SBIN.NS", start = start, end = "2019-04-01")
e['Daily Return'] = 0
sliced_data = e.loc[(e.index >= start) & (e.index <= end)]
year_avg = (sliced_data.iloc[-1]['Close'] - sliced_data.iloc[0]['Open'])*100 / sliced_data.iloc[0]['Open']
e.loc[sliced_data.index, 'Daily Return'] = year_avg
print(year_avg)
e.loc[sliced_data.index]['Daily Return']

51.47134667992929


Date
2016-04-01    51.471347
2016-04-04    51.471347
2016-04-05    51.471347
2016-04-06    51.471347
2016-04-07    51.471347
                ...    
2017-03-27    51.471347
2017-03-28    51.471347
2017-03-29    51.471347
2017-03-30    51.471347
2017-03-31    51.471347
Name: Daily Return, Length: 247, dtype: float64

In [25]:
eq = []
for i in range(5):
    start = "2016-04-01"
    end = str(int(start[:4]) + 3) + start[4:]
    e = pdr.get_data_yahoo(syms[i], start = start, end = end)
    eq.append(e)
    e['Daily Return'] = 0
    start = "2016-04-01"
    #e['Daily Return'] = (e['Close'] - e['Open'])*100 / e['Open']
    for j in range(3):
        end = str(int(start[:4]) + 1) + start[4:]
        sliced_data = e.loc[(e.index >= start) & (e.index <= end)]
        year_avg = (sliced_data.iloc[-1]['Close'] - sliced_data.iloc[0]['Open'])*100 / sliced_data.iloc[0]['Open']
        e.loc[sliced_data.index, 'Daily Return'] = year_avg
        start = end
        #print(e.loc[sliced_data.index, 'Daily Return'].tail(2))

#### Yearly Average Return

In [26]:
avg_return = []
for e in eq:
    ret = 0
    start = "2016-04-01"
    for i in range(3):
        end = str(int(start[:4]) + 1) + start[4:]
        sliced_data = e.loc[(e.index >= start) & (e.index <= end)]
        ret += (sliced_data.iloc[-1]['Close'] - sliced_data.iloc[0]['Open']) / sliced_data.iloc[0]['Open']
        start = end
    avg_return.append(ret*100/3)
yearly_avg_return = np.array(avg_return).reshape(-1, 1)
print(yearly_avg_return.shape, yearly_avg_return)

(5, 1) [[21.47260214]
 [21.24438064]
 [25.80360703]
 [ 3.5706382 ]
 [ 3.54023575]]


In [27]:
random_weights = np.random.dirichlet(np.ones(5), size = 1).T
excess_ret = yearly_avg_return * random_weights
print(random_weights)
print(excess_ret)
print(np.sum(excess_ret))

[[0.06185884]
 [0.06429266]
 [0.27952752]
 [0.17299986]
 [0.42132111]]
[[1.32827032]
 [1.36585765]
 [7.2128184 ]
 [0.61771992]
 [1.49157607]]
12.01624236592427


In [28]:
count = 0
for e in eq:
    e['Ret - avg'] = e['Daily Return'] - yearly_avg_return[count]
    count += 1

In [29]:
m = eq[0].shape[0]
n = len(eq)
excess_return = np.zeros([m, n])
for i in range(n):
    excess_return[:, i] = eq[i]['Ret - avg']

In [30]:
X = excess_return
mat = np.dot(np.transpose(X), X)

In [31]:
vcm = mat / m

In [32]:
std = []
count = 0
for e in eq:
    mean = yearly_avg_return[count]
    m = e.shape[0]
    st = sqrt(np.sum((e['Daily Return'] - mean)**2) / m)
    std.append(st)
    count += 1
print(std)

[27.62632166327554, 20.716227675044177, 50.05913640711804, 20.934839927246518, 37.22752483268484]


In [33]:
stds = np.array(std).reshape(-1, 1)
print(stds.shape)

(5, 1)


In [34]:
product_std = np.dot(stds, np.transpose(stds))
product_std.shape

(5, 5)

### Correlation
**corr = variance_covariance_matrix / product_of_sds**

In [35]:
corr = vcm / product_std
print(corr)

[[ 1.          0.31835789  0.76057095  0.76473878 -0.08952942]
 [ 0.31835789  1.         -0.37334073 -0.36735516 -0.97266612]
 [ 0.76057095 -0.37334073  1.          0.99997924  0.57855406]
 [ 0.76473878 -0.36735516  0.99997924  1.          0.57328625]
 [-0.08952942 -0.97266612  0.57855406  0.57328625  1.        ]]


In [36]:
srs = []
portfolio_vars = []
rand_wts = []
portfolio_return = []
risk_free_rate = 4
for i in range(0, 10000):
    random_weights = np.random.dirichlet(np.ones(5), size = 1).T
    rand_wts.append(random_weights)
    random_weighted_sd = stds * random_weights
    portfolio_std = np.sum(np.dot(np.dot(np.transpose(random_weighted_sd), corr), random_weighted_sd))
    portfolio_vars.append(sqrt(portfolio_std))
    excess_return = np.sum(yearly_avg_return * random_weights)
    portfolio_return.append(excess_return)
    sharpe_ratio = (excess_return - risk_free_rate) / portfolio_std
    srs.append(sharpe_ratio)
max_index = srs.index(max(srs))
best_wts = rand_wts[max_index]
max_sr = srs[max_index]
portfolio_var = portfolio_vars[max_index]
max_return = portfolio_return[max_index]
print(f"Max Sharpe Ratio: {max_sr}")
print(f"Best Weights: {best_wts}")
print(f"Portfolio Variance: {portfolio_var}, Sum of weights: {np.sum(best_wts)}")
print(f"Max Return: {max_return}")

Max Sharpe Ratio: 0.5175986260505181
Best Weights: [[0.01106247]
 [0.63111707]
 [0.02555543]
 [0.01892772]
 [0.31333732]]
Portfolio Variance: 4.709808374390654, Sum of weights: 1.0
Max Return: 15.481525375040803
